In [1]:
!pip install langchain-huggingface
!pip install langchain_community
!pip install chromadb
!pip install langchain-openai
!pip install openai
!pip install sentence-transformers
!pip install langchain
!pip install torch

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import chromadb

chroma_client = chromadb.Client()

In [4]:
collection = chroma_client.create_collection(name='tour')

In [5]:
import preprocess_functions as pf

documents = pf.make_csv_to_documents_with('전국 투어 데이터(숙박음식점제외).csv')

d:\SKN\3차 단위 프로젝트\rag관련\preprocess_functions.py:70: DtypeWarning: Columns (2,55,63,64,69,70,72,73,74,75,77,79,80,81,82,83) have mixed types. Specify dtype option on import or set low_memory=False.
  new_df = pd.read_csv(file_path)


In [6]:
print(documents[0])

page_content='광주의 사직단은 광주 양림동 광주광역시립사직도서관 위쪽 사직공원에 있는 조선시대의 제단이다. 사직단은 신성한 장소로, 사()는 땅의 신, 직()은 곡식의 신을 가리킨다. 사직단은 땅의 신과 곡식의 신에게 제사를 올리며 나라의 안녕과 풍요를 기원하는 사직제를 지내는 공간이다. 사직단은 종묘와 함께 신성시되어 종사()라고도 했다. 사직단은 서울뿐 아니라 각 군현에도 있었으며, 기곡제, 기우제, 기청제, 기설제 등을 지냈다. 광주 사직단 크기는 서울 사직단 규모(2장 5척의 정사각형, 높이 3척)의 반 정도이다. 갑오년 이후 폐지되었으며, 한말에는 사직단을 군사 훈련장으로 사용하기도 했었다. 1971년 동물원이 들어서면서 그 자취가 없어졌는데, 1993년 광주시가 사직단을 복원, 1994년 광주 향우회 회장(최기영)에 의해 100년 만에 사직제가 부활되었다. 국토와 오곡은 민생의 근본이므로 삼국 시대부터 국태민안을 기원하는 사직단을 설치하여 친히 왕이 나가서 제사를 올렸다. 지방에서는 성주가 제주가 되었으므로 광주 목사가 매년 제사를 주관하였다.' metadata={'place': '사직단', '홈페이지': 'https://tour.gwangju.go.kr/', '주소': '광주광역시 남구 사직길 49 (사동)', '이용시간': '상시 개방', '휴일': '연중무휴', '주차': '불가능', '화장실': '장애인 화장실 있음(사직공원 공중화장실)', '입장료': '무료', '접근로': '사직단까지 경사로가 설치되어 있음_무장애 편의시설'}


In [7]:
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA


# 텍스트 정보를 포함한 documents 리스트
text_documents = [doc.page_content for doc in documents]

# 텍스트 분할기 생성 (청킹할 크기 지정)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=0)

# 문서별로 청크 나누기
chunks = []
for doc in text_documents:
    chunks.extend(text_splitter.split_text(doc))

# 임베딩 모델
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# HuggingFaceEmbeddings 래퍼로 감싸기
embedding_function = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# 청크된 텍스트에 대해 임베딩 생성
embeddings = embedding_model.encode(chunks)

# 각 청크에 고유 ID 부여
ids = [f"chunk_{i}" for i in range(len(chunks))]

# 메타데이터는 원본 문서 정보에서 가져옴 (청크별로 메타데이터를 추가할 수 있음)
metadatas = []
for doc in documents:
    # 각 문서에 대해 청크 수만큼 메타데이터를 추가
    metadatas.extend([doc.metadata] * len(text_splitter.split_text(doc.page_content)))

# 배치 크기 설정
batch_size = 5000  # 최대 배치 크기

In [8]:
len(chunks)

46657

In [9]:
from langchain.schema import Document  # Document 객체 임포트

vectorstore = Chroma(
    collection_name="tour", 
    embedding_function=embedding_function,  # HuggingFaceEmbeddings 사용
    persist_directory='./chroma'
)

# 벡터 DB에 청크된 문서 추가 (배치 처리)
for i in range(0, len(chunks), batch_size):
    batch_chunks = chunks[i:i + batch_size]
    batch_embeddings = embeddings[i:i + batch_size]
    batch_metadatas = metadatas[i:i + batch_size]
    batch_ids = ids[i:i + batch_size]

    # 각 배치마다 Document 객체로 감싸기
    batch_documents = [Document(page_content=chunk, metadata=metadata) for chunk, metadata in zip(batch_chunks, batch_metadatas)]
    
    # 각 배치마다 벡터 DB에 추가
    vectorstore.add_documents(
        documents=batch_documents,
        embeddings=batch_embeddings,
        ids=batch_ids
    )
# 벡터 DB를 디스크에 저장
vectorstore.persist()

C:\Users\jenny\AppData\Local\Temp\ipykernel_9860\4125471535.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(
C:\Users\jenny\AppData\Local\Temp\ipykernel_9860\4125471535.py:26: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [1]:
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="place",
        description="관광지 장소의 이름.",
        type="string",
    ),
    AttributeInfo(
        name="홈페이지",
        description="관광지 장소의 공식 웹사이트.",
        type="string",
    ),
    AttributeInfo(
        name="주소",
        description="관광지 장소의 주소.",
        type="string",
    ),
    AttributeInfo(
        name="이용시간",
        description="관광지 장소가 개방되는 시간.",
        type="string",
    ),
    AttributeInfo(
        name="휴일",
        description="관광지 장소가 휴무인 날.",
        type="string",
    ),
    AttributeInfo(
        name="주차",
        description="관광지 장소의 주차 가능 여부에 대한 정보.",
        type="string",
    ),
    AttributeInfo(
        name="화장실",
        description="관광지의 화장실 시설에 대한 정보, 장애인 화장실 포함.",
        type="string",
    ),
    AttributeInfo(
        name="입장료",
        description="관광지 입장료에 대한 정보.",
        type="string",
    )
]

# 지역별 관광지 정보
region_info = [
    {"name": "서울", "details": "서울은 대한민국의 수도로, 고대와 현대가 조화를 이루는 도시입니다. 유명한 명소로는 경복궁, N서울타워, 명동 등이 있습니다."},
    {"name": "부산", "details": "부산은 대한민국 제2의 도시로, 해운대, 광안리 해변, 자갈치 시장 등 해양 관광이 유명합니다."},
    {"name": "대구", "details": "대구는 전통과 현대가 어우러진 도시로, 대구 근대골목, 팔공산 등이 유명하며, 매년 대구국제뮤지컬페스티벌이 열립니다."},
    {"name": "인천", "details": "인천은 수도권에 위치한 항구도시로, 인천 차이나타운과 송도 국제도시가 유명한 명소입니다."},
    {"name": "광주", "details": "광주는 한국의 문화와 예술의 중심지로, 광주비엔날레와 5.18 민주화운동 기념관이 주요 관광지입니다."},
    {"name": "대전", "details": "대전은 과학의 도시로 알려져 있으며, 대덕연구단지와 국립중앙과학관 등이 있습니다."},
    {"name": "울산", "details": "울산은 대한민국의 산업도시로, 현대자동차와 울산항이 유명하며, 울산 대공원이 인기 관광지입니다."},
    {"name": "세종", "details": "세종시는 대한민국의 행정수도로, 정부청사가 있는 도시이며, 자연 경관과 함께 살기 좋은 도시로 알려져 있습니다."},
    {"name": "경기", "details": "경기도는 서울을 둘러싸고 있는 지역으로, 수원 화성, DMZ 등 다양한 역사적 명소와 자연을 제공합니다."},
    {"name": "강원", "details": "강원도는 아름다운 자연경관을 자랑하는 지역으로, 설악산, 평창 등의 겨울 스포츠와 자연을 즐길 수 있습니다."},
    {"name": "충북", "details": "충청북도는 산과 호수가 많아 자연경관이 뛰어난 지역으로, 청주와 제천이 주요 도시입니다."},
    {"name": "충남", "details": "충청남도는 바다와 산이 어우러지는 지역으로, 서해안과 태안해안국립공원이 유명합니다."},
    {"name": "경북", "details": "경상북도는 역사적인 유산이 풍부한 지역으로, 경주와 안동, 팔공산 등이 유명한 관광지입니다."},
    {"name": "경남", "details": "경상남도는 바다와 산이 아름다운 곳으로, 통영, 거제도, 산청 등이 유명한 관광지입니다."},
    {"name": "전북", "details": "전라북도는 전통적인 문화와 자연이 어우러지는 지역으로, 전주한옥마을, 고창, 무주가 유명합니다."},
    {"name": "전남", "details": "전라남도는 아름다운 해변과 섬들이 많은 지역으로, 여수, 순천, 담양 등이 주요 관광지입니다."},
    {"name": "제주", "details": "제주도는 대한민국의 대표적인 관광지로, 한라산, 성산일출봉, 만장굴 등이 유명한 자연 경관을 자랑합니다."}
]

document_content = ""
for region in region_info:
    document_content += f"장소명: {region['name']}\n상세 설명: {region['details']}\n\n"

In [26]:
# LLM

from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

# 1. HuggingFace 모델을 사용한 임베딩 함수 설정
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"  # 사용한 모델 이름
embedding = HuggingFaceEmbeddings(model_name=embedding_model)

# HuggingFaceEmbeddings 래퍼로 감싸기
embedding_function = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# 2. Chroma 벡터 DB 로드
vectorstore = Chroma(
    collection_name="tour", 
    embedding_function=embedding_function,  # HuggingFaceEmbeddings 사용
    persist_directory='./chroma'            # 기존에 저장된 디렉토리에서 로드
)

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3. retriever 객체 생성
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    document_contents=document_content,
    metadata_field_info=metadata_field_info,
)

# 4. 사용자 질의 (예시)
query = "전남"

# 5. retriever로 유사한 문서 검색 -> results출력
results = retriever.get_relevant_documents(query)
print(results)
for doc in results:
    place = doc.metadata.get('place', 'Unknown Place')  # 'place' 값을 가져오거나, 없으면 'Unknown Place'로 설정
    address = doc.metadata.get('주소', 'Unknown Address')  # '주소' 값을 가져오거나, 없으면 'Unknown Address'로 설정
    print(f"Place: {place}, Address: {address}")



[Document(metadata={'휴일': '연중무휴', '입장료': '무료', '홈페이지': 'https://www.gunwi.go.kr/tour/', 'place': '화계서원', '주소': '대구광역시 군위군 의흥면 금양3길 16-3', '이용시간': '09:00~18:00', '주차': '가능 요금 (무료)', '화장실': '있음'}, page_content='지낸다.'), Document(metadata={'이용시간': '15:00~11:00', '화장실': '가능', '주소': '경기도 가평군 청평면 북한강로1636번길 133', '휴일': '연중무휴', '주차': '가능', 'place': '팜스테이글램핑', '이용요금': '- 글램핑 60,000원~160,000원 ※ 이용요금은 변동될 수 있으므로 홈페이지 참조 또는 전화 문의 요망', '홈페이지': 'http://www.farm-stay.co.kr/'}, page_content='점이 장점이다.'), Document(metadata={'주차 요금': '무료', '홈페이지': '▪ http://gimje.grandculture.net', '이용시간': '10:00~18:00', '주차': '주차 가능', '이용요금': '성인 3,000원 / 소인 2,000원', '주소': '전북특별자치도 김제시 만경읍 화포3길 54-9', '수용인원': '100명', '규모': '건평 462.812m² 지상 2층 규모', 'place': '성모암(김제)', '할인 정보': '1층카페 이용시 : 1,000원', '화장실': '있음(남녀 구분)', '휴일': '연중무휴'}, page_content='있는 암자입니다.'), Document(metadata={'휴일': '매주 화요일', '이용시간': '09:00~16:00 (휴게시간 11:15~12:45) ※ 겨울철 제설 작업 / 훈련 등으로 인하여 불시 출입통제 될 수 있음', '화장실': '있음', '주소': '경기도 연천군 신서면 동내로 1375-1', '입장

In [24]:
# 검색된 문서들을 하나의 문자열로 합쳐서 프롬프트에 전달
documents_text = ""
for result in results:
    # 결과에서 'place'와 'page_content'가 있는지 확인하고 추가
    place = result.metadata.get('place', 'Unknown Place')  # 'place'가 없으면 'Unknown Place'
    description = result.page_content if result.page_content else 'No description available.'  # 설명이 없으면 기본 텍스트 추가
    documents_text += f"장소: {place}\n설명: {description}\n\n"

# 프롬프트 템플릿
response_prompt = '''
당신은 여행 전문가입니다. 
다음 장소들을 바탕으로 {query}에 대해 친절하고 자세하게 추천을 해주세요.
{documents} 내용의 일부를 응답에 포함시켜 더 풍부한 답변을 생성해주세요.
답변을 가능한 한 세심하고 친절하게 작성해주세요. 각 여행지에 대한 간략한 설명도 포함해주세요. 
장소와 설명 구분: 검색된 문서에서 장소명과 설명을 구분하여 출력하고, 이를 바탕으로 여행지 추천을 제공해주세요.
모델은 각 장소에 대해 친절하게 여행지 추천 멘트를 생성하고, 이를 통해 사용자가 유용한 정보를 얻을 수 있도록 하세요.
results로 나온 장소들을 모두 포함하여 추천해주세요
'''

# 프롬프트 템플릿에 쿼리와 문서 텍스트 전달
prompt = response_prompt.format(query=query, documents=documents_text)

# LLM으로 응답 생성
response = llm.predict(prompt)

# 응답 출력
print("\nAI가 추천한 여행지:")
print(response)


AI가 추천한 여행지:
안녕하세요! 아이들과 함께 즐거운 여행을 계획하고 계신가요? 아래에 소개할 장소들은 가족 단위 여행에 적합하며, 아이들이 즐길 수 있는 다양한 활동과 경험을 제공합니다. 각 장소에 대한 간략한 설명과 함께 추천을 드리겠습니다.

### 1. 경희궁 숭정전
**설명:** 경희궁 숭정전은 서울의 역사적인 궁궐 중 하나로, 조선시대의 아름다운 건축물과 정원을 감상할 수 있는 곳입니다. 

**추천 멘트:** 경희궁 숭정전은 아이들에게 한국의 역사와 문화를 배울 수 있는 좋은 기회를 제공합니다. 궁궐 내에서 진행되는 다양한 프로그램이나 전통 의상 체험을 통해 아이들이 직접 조선시대의 왕과 왕비가 되어보는 재미를 느낄 수 있습니다. 또한, 근처에 위치한 서울역사박물관과 함께 방문하면, 역사에 대한 이해를 더욱 깊게 할 수 있습니다. 박물관에서는 다양한 전시와 체험 프로그램이 마련되어 있어 아이들이 흥미롭게 참여할 수 있습니다.

### 2. 한해랑 아트홀
**설명:** 한해랑 아트홀은 다양한 문화 행사와 전시가 열리는 공간으로, 예술과 문화를 가까이에서 체험할 수 있는 장소입니다.

**추천 멘트:** 한해랑 아트홀은 아이들과 함께 예술을 즐길 수 있는 멋진 공간입니다. 다양한 전시와 공연이 열리므로, 방문 시 아이들이 흥미를 느낄 만한 프로그램을 미리 확인해보세요. 예술 체험 프로그램도 종종 진행되니, 아이들이 직접 창작 활동에 참여해보는 것도 좋은 경험이 될 것입니다. 문화와 예술을 통해 아이들의 감성을 키워줄 수 있는 기회가 될 것입니다.

### 3. 청년스테이션
**설명:** 청년스테이션은 청년들을 위한 활력 공간으로, 다양한 프로그램과 활동이 진행되는 곳입니다.

**추천 멘트:** 청년스테이션은 아이들과 함께하는 가족 단위 방문에도 적합한 공간입니다. 다양한 워크숍이나 체험 프로그램이 마련되어 있어, 아이들이 새로운 것을 배우고 친구들과 소통할 수 있는 기회를 제공합니다. 또한, 청년들과의 소통을 통해 다양한 세대의 문화를 경험할 

In [ ]:
# sLLM
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.retrievers.self_query.base import SelfQueryRetriever


# 3. 자체 LLM 클래스를 정의하여 `kakaocorp/kanana-nano-2.1b-instruct` 사용
class KananaLLM:
    def __init__(self, model_name: str):
        self.model_name = model_name
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def _call(self, prompt: str) -> str:
        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
        with torch.no_grad():
            outputs = self.model.generate(inputs['input_ids'], max_length=1500, num_return_sequences=1, pad_token_id=self.tokenizer.eos_token_id)
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response

# LLM 설정 (kakaocorp/kanana-nano-2.1b-instruct 사용)
llm = KananaLLM(model_name="kakaocorp/kanana-nano-2.1b-instruct")

retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    document_contents=document_content,  # 문서 콘텐츠
    metadata_field_info=metadata_field_info  # 메타데이터 정보
)

# 5. 사용자 질의 예시
query = "강원도의 산 추천"

# 6. retriever로 유사한 문서 검색
results = retriever.get_relevant_documents(query)

# 7. 검색된 문서에서 place와 주소 출력
for doc in results:
    place = doc.metadata.get('place', 'Unknown Place')  # 'place' 값을 가져오거나, 없으면 'Unknown Place'로 설정
    address = doc.metadata.get('주소', 'Unknown Address')  # '주소' 값을 가져오거나, 없으면 'Unknown Address'로 설정
    print(f"Place: {place}, Address: {address}")

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 모델과 토크나이저 로드
model_name = "kakaocorp/kanana-nano-2.1b-instruct"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 검색된 문서들을 하나의 문자열로 합쳐서 프롬프트에 전달
documents_text = ""
for result in results:
    # 결과에서 'place'와 'page_content'가 있는지 확인하고 추가
    place = result.metadata.get('place', 'Unknown Place')  # 'place'가 없으면 'Unknown Place'
    description = result.page_content if result.page_content else 'No description available.'  # 설명이 없으면 기본 텍스트 추가
    documents_text += f"장소: {place}\n설명: {description}\n\n"

# 프롬프트 템플릿
response_prompt = '''
당신은 여행 전문가입니다. 
다음 장소들을 바탕으로 {query}에 대해 친절하고 자세하게 추천을 해주세요.
{documents} 내용의 일부를 응답에 포함시켜 더 풍부한 답변을 생성해주세요.
답변을 가능한 한 세심하고 친절하게 작성해주세요. 각 여행지에 대한 간략한 설명도 포함해주세요. 
장소와 설명 구분: 검색된 문서에서 장소명과 설명을 구분하여 출력하고, 이를 바탕으로 여행지 추천을 제공해주세요.
모델은 각 장소에 대해 친절하게 여행지 추천 멘트를 생성하고, 이를 통해 사용자가 유용한 정보를 얻을 수 있도록 하세요.
3가지의 장소를 추천해주세요
'''

# 프롬프트 템플릿에 쿼리와 문서 텍스트 전달
prompt = response_prompt.format(query=query, documents=documents_text)

# 입력 텍스트 토크나이징
inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)

# 모델로부터 응답 생성
with torch.no_grad():
    outputs = model.generate(inputs['input_ids'], max_length=1500, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

# 출력 텍스트 디코딩
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# 응답 출력
print("\nAI가 추천한 여행지:")
print(response)

OSError: 이 작업을 완료하기 위한 페이징 파일이 너무 작습니다. (os error 1455)